# Рекомендация тарифов

**Описание проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Цель проекта:**

Построить модель с максимально большим значением accuracy, довести долю правильных ответов минимум до 0.75. Проверить accuracy на тестовой выборке.

## Открыть и изучить файл

Импортируем все необходимые библиотеки и структуры данных

In [86]:
!pip install -U fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
import pandas as pd
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [88]:
!wget https://code.s3.yandex.net/datasets/users_behavior.csv

--2023-02-27 04:23:08--  https://code.s3.yandex.net/datasets/users_behavior.csv
Resolving code.s3.yandex.net (code.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to code.s3.yandex.net (code.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134358 (131K) [application/octet-stream]
Saving to: ‘users_behavior.csv.2’

users_behavior.csv. 100%[===================>] 131.21K  --.-KB/s    in 0.1s    

2023-02-27 04:23:08 (1.02 MB/s) - ‘users_behavior.csv.2’ saved [134358/134358]



In [89]:
users_behavior = pd.read_csv("users_behavior.csv")

In [90]:
users_behavior.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [91]:
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [92]:
users_behavior['messages']=users_behavior['messages'].astype("int")

In [93]:
users_behavior.duplicated().sum()

0

In [94]:
users_behavior.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

In [95]:
users_behavior.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


# **Вывод**

1. Перед нами DataFrame с 5 колонками и 3214 строчками
2. Преобразовали тип данных в столбце 'messages' в 'Int'
3. С регистром все в порядке 
4. Дубликатов в DataFrame нет

## Разбить данные на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки.

In [96]:
features_train, target_train, features_valid, target_valid, features_test, target_test = train_valid_test_split(
    users_behavior, target='is_ultra', train_size=0.60, valid_size=0.2, test_size=0.2)

In [97]:
features_train.shape

(1928, 4)

In [98]:
target_train.shape

(1928,)

In [99]:
features_valid.shape

(643, 4)

In [100]:
target_valid.shape

(643,)

In [101]:
features_test.shape

(643, 4)

In [102]:
target_test.shape

(643,)

## Исследовать модели

Исследуем качество разных моделей, изменяя гиперпараметры. 

Начнем с исследования модели Random Forest.

### Random Forest

In [103]:
best_model_random_forest = None
best_accuracy_random_forest = 0
best_est_random_forest = 0
best_depth_random_forest = 0
for est in range(2,51,10):
    for depth in range(2,50):
        model_RF = RandomForestClassifier(random_state = 123456, n_estimators = est, max_depth = depth, n_jobs=-1)
        model_RF.fit(features_train,target_train)
        prediction_valid_random_forest = model_RF.predict(features_valid)
        accuracy_random_forest = accuracy_score(target_valid, prediction_valid_random_forest)
        if best_accuracy_random_forest < accuracy_random_forest:
            best_model_random_forest = model_RF
            best_depth_random_forest = depth
            best_est_random_forest = est
            best_accuracy_random_forest = accuracy_random_forest

In [104]:
print("Accuracy Random Forest:", best_accuracy_random_forest, "количество деревьев", best_est_random_forest, "глубина дерева", best_depth_random_forest )

Accuracy Random Forest: 0.80248833592535 количество деревьев 32 глубина дерева 7


Исследуем модель Decision Tree

In [105]:
best_decision_tree = None
best_accuracy_decision_tree = 0
best_depth_decision_tree = 0
for depth in range(2,51):
    model_DF = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model_DF.fit(features_train,target_train)
    prediction_valid_decision_tree = model_DF.predict(features_valid)
    accuracy_decision_tree = accuracy_score(prediction_valid_decision_tree, target_valid)
    if best_accuracy_decision_tree < accuracy_decision_tree:
        best_decision_tree = model_DF
        best_accuracy_decision_tree = accuracy_decision_tree
        best_depth_decision_tree = depth

In [106]:
print("Accuracy Decision Tree", best_accuracy_decision_tree, "глубина дерева:", best_depth_decision_tree )

Accuracy Decision Tree 0.7838258164852255 глубина дерева: 3


Исследуем модель Logistic Regression

In [107]:
model_logistic_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_logistic_regression.fit(features_train, target_train)
prediction_valid_logistic_regression = model_logistic_regression.predict(features_valid)
accuracy_logistic_regression = accuracy_score(prediction_valid_logistic_regression, target_valid)

In [108]:
print("Accuracy Logistic Regression:", accuracy_logistic_regression)

Accuracy Logistic Regression: 0.7340590979782271


# **Вывод**

Наиболее высокий результат предсказаний делает модель Random Forest, это видно по accuracy на валидационной выборке.

## Проверить модель на тестовой выборке

Проверим модель на тестовой выборке.

In [109]:
best_model_random_forest.fit(
    features_test,
    target_test
)
prediction_test_random_forest = best_model_random_forest.predict(features_test)
accuracy_test_random_forest = accuracy_score(target_test, prediction_test_random_forest)
print("Accuracy на тестовой выборке", accuracy_test_random_forest)

Accuracy на тестовой выборке 0.8709175738724728


In [110]:
best_decision_tree.fit(
    features_test,
    target_test
)
prediction_test_decision_tree = best_decision_tree.predict(features_test)
accuracy_test_decision_tree = accuracy_score(target_test, prediction_test_decision_tree)
print("Accuracy на тестовой выборке", accuracy_test_decision_tree)

Accuracy на тестовой выборке 0.8087091757387247


In [111]:
model_logistic_regression.fit(
    features_test,
    target_test
)
prediction_test_logistic_regression = model_logistic_regression.predict(features_test)
accuracy_logistic_regression_test = accuracy_score(target_test, prediction_test_logistic_regression)
print("Accuracy на тестовой выборке", accuracy_logistic_regression_test)

Accuracy на тестовой выборке 0.7433903576982893


Наиболее высокий accuracy на тестовой выборке делает Random Forest.

Объединим данные (обучающие и валидационные) и проверим accuracy.

In [112]:
features_full_train = pd.concat([features_train, features_valid])
target_full_train = pd.concat([target_train, target_valid])

In [113]:
best_model_random_forest.fit(features_full_train, target_full_train)

RandomForestClassifier(max_depth=7, n_estimators=32, n_jobs=-1,
                       random_state=123456)

In [114]:
prediction_full_random_forest = best_model_random_forest.predict(features_test)

In [115]:
accuracy_full_random_forest = accuracy_score(target_test, prediction_full_random_forest)
print("Accuracy на объединенной выборке", accuracy_full_random_forest)

Accuracy на объединенной выборке 0.807153965785381


In [116]:
best_decision_tree.fit(features_full_train,target_full_train)

DecisionTreeClassifier(max_depth=3, random_state=12345)

In [117]:
prediction_full_decision_tree = best_decision_tree.predict(features_test)

In [118]:
accuracy_full_decision_tree = accuracy_score(target_test, prediction_full_decision_tree)
print("Accuracy на объединенной выборке", accuracy_full_decision_tree)

Accuracy на объединенной выборке 0.7853810264385692


In [119]:
model_logistic_regression.fit(features_full_train, target_full_train)

LogisticRegression(max_iter=1000, random_state=12345)

In [120]:
prediction_full_logistic_regression = model_logistic_regression.predict(features_test)

In [121]:
accuracy_full_logistic_regression = accuracy_score(target_test, prediction_full_logistic_regression)
print("Accuracy на объединенной выборке", accuracy_full_logistic_regression)

Accuracy на объединенной выборке 0.702954898911353


## (бонус) Проверить модели на адекватность

In [122]:
dummy_prediction_1 = [1 for i in range(643)]

In [123]:
dummy_accuracy_score_1 = accuracy_score(dummy_prediction_1, target_test) 
dummy_accuracy_score_1

0.30326594090202175

In [124]:
dummy_prediction_0 = [0 for i in range(643)]

In [125]:
dummy_accuracy_score_0 = accuracy_score(dummy_prediction_0, target_test) 
dummy_accuracy_score_0

0.6967340590979783

## Вывод

Изучены исходные данные. Исходные данные разделены на обучающую, валидационную и тестовую выборки. Исследовано качество разных моделей, с различными гиперпараметрами. 

1.   Наиболее высокий результат предсказаний делает модель Random Forest, это видно по accuracy на валидационной выборке.
2.   Проверено качество модели на тестовой выборке. Наиболее высокий accuracy на тестовой выборке делает Random Forest.
3.  Проверено качество модели на объединенной выборке валидационная и тестовая. Наиболее высокий accuracy на этой выборке делает Random Forest.
4.  Модель проверена на вменяемость. Аccuracy_score для "глупой модели" оказался ниже, чем у лучшей модели, значит модель обладает предиктивной силой и может считаться адекватной.